# **EV Cars Dataset**
### Dhivya Murugan

Let's load in data about the Vehicle Fuel Type by Zip Code, selecting for 'battery electric' and 'plug in hybrid' types.

In [ ]:
import pandas as pd

In [ ]:
url = 'https://data.ca.gov/dataset/15179472-adeb-4df6-920a-20640d02b08c/resource/d304108a-06c1-462f-a144-981dd0109900/download/vehicle-fuel-type-count-by-zip-code.csv'
raw_data = pd.read_csv(url, low_memory=False)
ev_types = ['Battery Electric', 'Plug-in Hybrid']
ev_data = raw_data[raw_data.Fuel.isin(ev_types)]

In [ ]:
ev_data # for visualization

Let's draw a chloropleth of EV users in LA County! For something fun to look at, I guess. I'm sure we'll need chloropleths later. 

In [ ]:
import folium
import json
from folium import plugins
import urllib.request

url = 'https://raw.githubusercontent.com/ritvikmath/StarbucksStoreScraping/master'
la_map_url = f'{url}/laMap.geojson'
la_zips_url = f'{url}/laZips.geojson'

In [ ]:
with urllib.request.urlopen(la_map_url) as url:
    la_area = json.loads(url.read().decode())

In [ ]:
carDist = ev_data.groupby('Zip Code')['Vehicles'].sum().drop(labels=['OOS', 'Other']).reset_index(name='Vehicles')
la_map = folium.Map(location=[34.0522,-118.2437], tiles='Stamen Toner', zoom_start=9)
folium.Choropleth(geo_data=la_zips_url, data=carDist, columns=['Zip Code', 'Vehicles'], 
                 key_on='feature.properties.zipcode', fill_color='YlGn', fill_opacity=1).add_to(la_map)

In [ ]:
print(carDist)

In [ ]:
la_map # for visualization

Let's also visualize where the chargers are in LA, just because we can. 


In [ ]:
url = 'https://raw.githubusercontent.com/medhivya/ev-charger/main/Chargepoint%20Session-Details-Summary-20200612.csv'
cp_data_raw = pd.read_csv(url, usecols=['Latitude', 'Longitude', 'Driver Postal Code'])
cp_data = cp_data_raw.drop_duplicates()

In [ ]:
# from NREL
url = 'https://developer.nrel.gov/api/alt-fuel-stations/v1.csv?fuel_type=ELEC&state=CA&status=E&access=public&api_key=JDCZwYUVGgKuaIHzmjwrmYsCaKbXotIHg5dyr3bv'
nrel_chargers_data_raw = pd.read_csv(url, usecols=['City', 'Latitude', 'Longitude'])
nrel_chargers_data_la = nrel_chargers_data_raw[nrel_chargers_data_raw.City == 'Los Angeles']

# drops an errorneous charger point
nrel_chargers_data_la = nrel_chargers_data_la.sort_values(by=['Longitude'])
nrel_chargers_data_la.drop(nrel_chargers_data_la.tail(1).index,inplace=True)

cp_data = pd.concat([nrel_chargers_data_la,cp_data]).drop_duplicates() # concatenate NREL data and ChargerPoint data

In [ ]:
la_map_cp = folium.Map(location=[34.0522,-118.2437], tiles='Stamen Toner', zoom_start=9)
folium.GeoJson(la_area).add_to(la_map_cp)
for i,row in cp_data.iterrows():
    folium.CircleMarker((row.Latitude,row.Longitude), radius=1, weight=2, color='red', fill_color='red', fill_opacity=.5).add_to(la_map_cp)

In [ ]:
la_map_cp # for visualization

Ok, this one's a little ambitious - but now that we've visualized where the charging ports are, let's visualize how far people have come to charge their vehicles. Let's take that from the driver zip code in the chargepoint data set. 

In [ ]:
!pip install mpu
import mpu
import numpy as np
import pandas as pd

In [ ]:
zip_code_coords_url = 'https://raw.githubusercontent.com/medhivya/ev-charger/main/ZIP_Codes_and_Postal_Cities.csv'
raw_lac_zips = pd.read_csv(zip_code_coords_url, low_memory=False, usecols=['Location']).Location.tolist()
cleaner_zips = {int(row[:5]):tuple([float(val) for val in row[6:].strip(')').split(',')]) for row in raw_lac_zips}

def get_haversine(row):
  curr = tuple((row.Latitude, row.Longitude))
  return round(mpu.haversine_distance(curr, cleaner_zips.get(int(row['Driver Postal Code']), curr)) * 1000, 0)

cp_data_rad = cp_data_raw
cp_data_rad['Driver Postal Code'] = pd.to_numeric(cp_data_rad['Driver Postal Code'], errors='coerce')
cp_data_rad['Radius'] = cp_data_rad.dropna().apply(get_haversine, axis=1)
# print(cp_data_rad[cp_data_rad.Latitude == 33.797703])
cp_data_rad = cp_data_rad.groupby(['Latitude', 'Longitude'])['Radius'].mean().reset_index()
cp_data_rad['Radius'].mask(cp_data_rad['Radius'] == 0.0, 1000.0, inplace=True)  # this is a default, maybe we want to change this

In [ ]:
cp_data_rad

In [ ]:
(np.median(cp_data_rad['Radius']))

10473.895059103463

In [ ]:
(np.median(cp_data_rad['Radius'])) / 1609.344  # this is a meter to mile conversion, to validate radius that we will use to characterize a charger

6.50817666024384

The radius for each charger is approximately 6.5 miles

In [ ]:
la_map_rad = folium.Map(location=[34.0522,-118.2437], tiles='Stamen Toner', zoom_start=9)
folium.GeoJson(la_area).add_to(la_map_rad)
for i,row in cp_data_rad.iterrows():
    folium.Circle((row.Latitude,row.Longitude), radius=row.Radius, weight=2, color='red', fill_color='red', fill_opacity=.05).add_to(la_map_rad)

In [ ]:
la_map_rad #for visualization

# **Census Data Download and Analysis**
### Dan Rabinovich

In this section, I download data from the five-year US Census estimates for the Los Angeles county. I analyze the data to calculate the Concentrated Disadvantage Index and the expected number of vehicles per household, both by census tract. 


Here are the elements of the CDI:

1.   Percent of individuals below the poverty line

> * Poverty Status in 2018 for *Children Under 18*: A13003A
>> * Population Under 18 Years of Age for Whom Poverty Status Is Determined: A13003A_001
>> * Living in Poverty: A13003A_002

> * Poverty Status in 2018 for Population *Age 18 to 64*: A13003B
>> * Population Age 18 to 64 for Whom Poverty Status  Is Determined: A13003B_001
>> * Living in Poverty: A13003B_002

> * Poverty Status in 2018 for Population *Age 65 and Over*: A13003C
>> * Population Age 65 and Over for Whom Poverty  Status Is Determined: A13003C_001
>> * Living in Poverty: A13003C_002

> *(Total # living in poverty) / (Total # for whom poverty status is determined)*

2.   Percent of individuals on public assistance

> * Households with Public Assistance Income: A10014
>> * Households: A10014_001
>> * Households with Public Assistance Income: A10014_002


3.   Percent female-headed households

> * Households by household type: A10008
>> * Households: A10008_001
>> * Female Householder, No Husband Present: A10008_006
>> * Nonfamily households, female householder: A10008_009


4.   Percent unemployed

> * Employment Status for Total Population 16 Years and Over: A17002
>> * Population 16 Years and Over: A17002_001
>> * Unemployed: A17002_006


5.   Percent less than age 18

> * Age: A01001
>> * Total Population: A01001_001
>> * Under 5 Years: A01001_002
>> * 5 to 9 Years: A01001_003
>> * 10 to 14 Years: A01001_004
>> * 15 to 17 Years: A01001_005


Here is the other census information I want to  download:

Travel Time to Work for Workers 16 Years and Over: A09001
>   Contains workers age 16 and over, numbers for those who do or do not work at home, and ten minute commute time intervals up to 90 minutes

Average Commute to Work (In Min): A09003
>  Self-explanatory

Housing Units by Vehicle Available: A10030 
>  May be helpful in determining need. More cars means more chargers, right?

Vehicles Available (Renter-Occupied Housing Units): A10054B
>  Same as above

https://www.socialexplorer.com/tables/ACS2018_5yr/R12681169

## Remove Extras


In [ ]:
import pandas as pd

In [ ]:
def remove_fips_extras(df): # To isolate census tracts in City of LA
  la_tracts_df_url = 'https://github.com/d-rabinovich/EV-Charger-Project/blob/main/city_of_la_tracts.csv?raw=true'
  la_tracts_df = pd.read_csv(la_tracts_df_url, encoding = "ISO-8859-1")
  tract_ids = la_tracts_df['TRACTCE10']
  tract_ids = ['6037' + str(elem) for elem in tract_ids]

  indices = []
  for ind, row in df.iterrows():
    if str(int(row['Geo_FIPS'])) in tract_ids:
      indices.append(ind)
  
  ret_df = df.iloc[indices]
  return ret_df

def remove_zip_extras(df): # To isolate zip codes in City of LA
  la_zips_df_url = 'https://github.com/d-rabinovich/EV-Charger-Project/blob/main/Los_Angeles_City_Zip_Codes.csv?raw=true'
  la_zips_df = pd.read_csv(la_zips_df_url, encoding = "ISO-8859-1")
  zipcodes = la_zips_df['ZIP']

  indices = []
  for ind, row in df.iterrows():
    if row['Geo_FIPS'] in zipcodes:
      indices.append(ind)
  
  ret_df = df.iloc[indices]
  return ret_df

## Concentrated Disadvantage

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# Input is raw census numbers, output is statistics as percentages as well as FIPS
def aggregate_cdi_data(url):
  # import file
  df_census = pd.read_csv(url, encoding = "ISO-8859-1")
  # choose desired columns, update array
  cols_to_use = ['SE_A13003A_001', 'SE_A13003A_002',                # Children below poverty line
                 'SE_A13003B_001', 'SE_A13003B_002',                # 18 - 64 below poverty line
                 'SE_A13003C_001', 'SE_A13003C_002',                # >65 below poverty line
                 'SE_A10014_001', 'SE_A10014_002',                  # Individuals on Public assistance
                 'SE_A10008_001', 'SE_A10008_006', 'SE_A10008_009', # Female-headed households
                 'SE_A17002_001', 'SE_A17002_006',                  # Percent unemployed
                 'SE_B01001_001', 'SE_B01001_002',                  # Percent less than age 18
                 'Geo_FIPS', 'Geo_TRACT']                           # Location information

  df_census = df_census[cols_to_use]
  # df_census = remove_fips_extras(df_census)
  
  # 1. Percent of individuals below the poverty line
  population_cols = ['SE_A13003A_001', 'SE_A13003B_001', 'SE_A13003C_001']
  poverty_cols = ['SE_A13003A_002', 'SE_A13003B_002', 'SE_A13003C_002']
  df_census['%_poverty'] = df_census[poverty_cols].sum(axis=1) / df_census[population_cols].sum(axis=1)
  df_census.drop(columns = (population_cols + poverty_cols), inplace = True)

  # 2. Percent of individuals on public assistance
  df_census['%_assistance'] = df_census['SE_A10014_002'] / df_census['SE_A10014_001']
  df_census.drop(columns = ['SE_A10014_001', 'SE_A10014_002'], inplace = True)

  # 3. Percent of female-headed households
  f_house_cols = ['SE_A10008_006', 'SE_A10008_009']
  df_census['%_f_households'] = df_census[f_house_cols].sum(axis = 1) / df_census['SE_A10008_001']
  df_census.drop(columns = ['SE_A10008_001', 'SE_A10008_006', 'SE_A10008_009'], inplace = True)

  # 4. Percent unemployed
  df_census['%_unemployed'] = df_census['SE_A17002_006'] / df_census['SE_A17002_001']
  df_census.drop(columns = ['SE_A17002_001', 'SE_A17002_006'], inplace = True)

  # 5. Percent less than age 18
  df_census['%_children'] = df_census['SE_B01001_002'] / df_census['SE_B01001_001']
  df_census.drop(columns = ['SE_B01001_001', 'SE_B01001_002'], inplace = True)

  return df_census

In [ ]:
df_census = aggregate_cdi_data('https://github.com/d-rabinovich/EV-Charger-Project/blob/main/raw_census_data.csv?raw=true')

In [ ]:
# Function which calculates average z-score for a census tract
# In other words, calculates disadvantage index
def z_score_calculator(df_cdi):
  cols_to_use = ['%_poverty', '%_assistance', '%_f_households',
                 '%_unemployed', '%_children']

  # Calculate Z-scores
  for col in df_cdi.columns:
    if col == 'Geo_FIPS' or col == 'Geo_TRACT': continue # ignore location ids
    df_cdi[col] = (df_cdi[col] - df_cdi[col].mean())/df_cdi[col].std(ddof=0)
    
  # Average the z-scores, delete old rows
  df_cdi['r_concentrated_disadvantage'] = df_cdi.iloc[:, 2:6].mean(axis=1)
  df_cdi.drop(columns = cols_to_use, inplace = True)

  # forgive Dhivya as she tries to fix what she thinks is wrong
  df_cdi['Geo_FIPS'] = ['0' + str(elem) for elem in df_cdi['Geo_FIPS']]
  
  print(df_cdi.isna().sum())
  return df_cdi

In [ ]:
df_cdi = z_score_calculator(df_census)

Geo_FIPS                        0
Geo_TRACT                       0
r_concentrated_disadvantage    19
dtype: int64


## Mapping Concentrated Disadvantage

I seem to have some outliers. I will code up a function to remove them 

In [ ]:
# input is dataframe, col to remove on, and percent to remove on each side
# returns dataframe with margins removed
def remove_outliers(df, col_name, margin): 
  middle_portion = df[col_name].between(df[col_name].quantile(margin), df[col_name].quantile(1 - margin))
  df.drop(df[~middle_portion].index, inplace = True)
  return df

I'm gonna try to make a choropleth using Dhivya's code as an example.

In [ ]:
import folium
import json
from folium import plugins

la_tracts_url = 'http://boundaries.latimes.com/1.0/boundary-set/census-tracts-2012/?format=geojson'

In [ ]:
df_cdi_copy = remove_outliers(df_census_filled, 'r_concentrated_disadvantage', .01)

In [ ]:
la_map = folium.Map(location=[34.0522,-118.2437], tiles='Stamen Toner', zoom_start=9)
folium.Choropleth(
    geo_data=la_tracts_url, 
    data=df_cdi_copy, 
    columns=['Geo_FIPS', 'r_concentrated_disadvantage'], 
    key_on='feature.properties.name', 
    fill_color='BuPu', 
    fill_opacity=1,
    line_opacity=.2
).add_to(la_map)

In [ ]:
la_map

## Other Census Data

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
def aggregate_other_census_data(url):
  df_census = pd.read_csv(url, encoding = "ISO-8859-1")

  # First, calculate total cars
  household_car_cols = ['SE_A10030_003', 'SE_A10030_004', 'SE_A10030_005', 'SE_A10030_006', 'SE_A10030_007']
  car_counts_list = []
  for ind, row in df_census.iterrows():
    car_counts = []
    for i in range(len(household_car_cols)):
      car_counts.append((i + 1) * row[household_car_cols[i]]) 
    car_counts_list.append(np.nansum(np.array(car_counts)))
  df_census['a_total_cars'] = car_counts_list

  new_relative_labels = ['r_commute_time', 'r_median_household_income', 'r_average_household_size', 
                         'r_population_density', 'r_median_house_value']
  new_absolute_labels = ['a_total_population', 'a_pop_below_18', 'a_pop_between_18_34', 'a_pop_between_34_65',
                         'a_pop_over_65', 'a_occupied_households', 'a_family_households', 'a_nonfamily_households',
                         'a_householder_less_than_high_school', 'a_householder_high_school_degree', 'a_householder_some_college',
                         'a_householder_bachelors_degree', 'a_older_than_3_in_school', 'a_older_than_3_not_in_school',
                         'a_not_high_school_grad', 'a_high_school_grad', 'a_employed', 'a_unemployed', 'a_not_in_labor_force',
                         'a_30_49_percent_income_to_rent', 'a_over_50_percent_income_to_rent', 'a_commuters_by_car_truck_van',
                         'a_commuters_by_car_truck_van_alone', 'a_commuters_by_car_truck_van_carpool', 'a_children_living_with_single_parents',
                         'a_private_sector', 'a_public_sector', 'a_self_employed', 'a_private_non-profit', 'a_unpaid_family_workers',
                         'a_households_social_security_income', 'a_households_SsI', 'a_households_retirement_income', 'a_housing_units',
                         'a_vacant_units','a_agr_industry', 'a_construct_industry', 'a_manufact_industry', 'a_wholesale_industry',
                         'a_retail_industry', 'a_transport_industry', 'a_info_industry','a_finance_industry','a_prof_mgmt_industry',
                         'a_education_industry', 'a_arts_industry','a_public_admin_industry', 'a_other_service_industry', 'a_mgmt_occ',
                         'a_professional_occ', 'a_health_occ', 'a_protecc_occ', 'a_food_occ','a_cleaning_occ', 'a_pers_care_occ',
                         'a_sales_occ', 'a_office_occ', 'a_farming_occ', 'a_construct_occ', 'a_product_occ', 'a_transport_occ' ]

  og_relative_labels = ['SE_A09003_001', 'SE_A14006_001', 'SE_A10003_001', 'SE_A00002_002', 'SE_A10036_001']

  og_absolute_labels = ['SE_B01001_001', 'SE_B01001_002', 'SE_B01001_003', 'SE_B01001_004', 'SE_B01001_005',
                        'SE_A10030_001', 'SE_A10008_002', 'SE_A10008_007', 'SE_A10021_002', 'SE_A10021_003',
                        'SE_A10021_004', 'SE_A10021_005', 'SE_A12004_002', 'SE_A12004_003', 'SE_A12003_002',
                        'SE_A12003_003', 'SE_A17002_005', 'SE_A17002_006', 'SE_A17002_007', 'SE_B18002_002',
                        'SE_B18002_003', 'SE_A09005_002', 'SE_A09005_009', 'SE_A09005_010', 'SE_A10065_002',
                        'SE_A17009_002', 'SE_A17009_003', 'SE_A17009_004', 'SE_A17009_005', 'SE_A17009_006',
                        'SE_A10017_002', 'SE_A10018_002', 'SE_A10015_002', 'SE_A10044_001', 'SE_A10044_003', 
                        'SE_A17004_002', 'SE_A17004_003', 'SE_A17004_004', 'SE_A17004_005', 'SE_A17004_006',
                        'SE_A17004_007', 'SE_A17004_008', 'SE_A17004_009', 'SE_A17004_010', 'SE_A17004_011',
                        'SE_A17004_012', 'SE_A17004_013', 'SE_A17004_014', 
                        'SE_B17008_002', 'SE_B17008_003',
                        'SE_B17008_004', 'SE_B17008_005', 'SE_B17008_006', 'SE_B17008_007', 'SE_B17008_008',
                        'SE_B17008_009', 'SE_B17008_010', 'SE_B17008_011', 'SE_B17008_012', 'SE_B17008_013', 
                        'SE_B17008_014']
  
  for ind in range(len(new_relative_labels)):
    df_census[new_relative_labels[ind]] = df_census[og_relative_labels[ind]]
  
  for ind in range(len(new_absolute_labels)):
    df_census[new_absolute_labels[ind]] = df_census[og_absolute_labels[ind]]
  
  for label, col in df_census.iteritems():
    if label[0:2] == 'SE' or (label[0] == 'G' and (label != 'Geo_FIPS' and label != 'Geo_TRACT')):
      df_census.drop(columns=label, inplace=True)

  df_census['Geo_FIPS'] = ['0' + str(elem) for elem in df_census['Geo_FIPS']]
  return df_census

In [ ]:
df_census = aggregate_other_census_data('https://github.com/d-rabinovich/EV-Charger-Project/blob/main/raw_census_data.csv?raw=true')

[2829, 2671, 3136, 2171, 3223, 2751, 1366, 2433, 1148, 2979, 2073, 3221, 4284, 3142, 5140, 2159, 2901, 2614, 2542, 2314, 2736, 1725, 2428, 2681, 1674, 1620, 1533, 1606, 1792, 1860, 1543, 1240, 2612, 2989, 1769, 1192, 2194, 4020, 3269, 3797, 2309, 3120, 2120, 2519, 3720, 1184, 1671, 3548, 3569, 2409, 2515, 1791, 2357, 3715, 2847, 2253, 2208, 1737, 1847, 2998, 1761, 2641, 2603, 1339, 4016, 5192, 1723, 2370, 2073, 2448, 2110, 3130, 2883, 1997, 3136, 3588, 2235, 2494, 3703, 4942, 2723, 3501, 4536, 3347, 4956, 2637, 2467, 3030, 2565, 3383, 1815, 3022, 3917, 2466, 1324, 2965, 2269, 2815, 4170, 3077, 4067, 3556, 4039, 3880, 66, 3037, 4735, 3824, 3266, 1966, 3439, 2045, 1806, 3038, 1852, 2068, 1802, 2891, 3219, 2617, 3310, 1171, 1208, 1341, 1639, 1643, 3011, 2519, 4650, 2330, 2479, 2253, 2366, 1232, 1567, 1486, 3169, 3303, 3572, 2822, 1584, 1657, 888, 1026, 876, 1500, 1405, 1314, 1925, 3156, 3947, 2675, 2255, 1741, 1752, 1722, 1374, 2431, 1792, 1711, 1735, 2215, 3645, 2317, 1513, 1197, 2013, 1

In [ ]:
# print(df_census.shape)
# print(df_cdi.shape)

## Combine the Two

In [ ]:
def combine_census_data(df_cdi, df_other):
  cols_to_drop = ['Geo_FIPS', 'Geo_TRACT']
  df_other.drop(columns = cols_to_drop, inplace = True)
  df_census = pd.concat([df_cdi, df_other], axis=1)
  return df_census

In [ ]:
df_census_copy = df_census.copy()
df_census_missing = combine_census_data(df_cdi, df_census_copy)
# print(census_data.iloc[0])
# print(df_census.iloc[0])
# print(df_census.columns)
# print(df_census_missing.columns)

## Census Data Preprocessing

* First, fill na values as mean of neighbors' values
* Forward fill the rest


In [ ]:
import pandas as pd
from shapely.geometry import Point
from shapely.geometry import Polygon
from shapely.geometry import MultiPolygon
import numpy as np
import geopy
import json
from urllib.request import urlopen
import time

In [ ]:
la_tracts_url = 'http://s3-us-west-2.amazonaws.com/boundaries.latimes.com/archive/1.0/boundary-set/census-tracts-2012.geojson'
response = urlopen(la_tracts_url)
la_tracts = json.loads(response.read())

In [ ]:
tracts_dict = {} # Dictionary of 'tract id' : list of coordinates
for obj in la_tracts["features"]:
  tract_id = "06037" + str(obj['properties']['metadata']['TRACTCE'])
  tracts = []
  if obj['geometry']['type'] == "MultiPolygon":
    for polyg in obj['geometry']['coordinates']:
      tracts.append(Polygon(polyg[0]))
      tracts.append(polyg[0])
  else:
    tracts.append(Polygon(obj['geometry']['coordinates'][0]))
    print(Polygon(obj['geometry']['coordinates'][0]))
  tracts_dict[tract_id] = tracts

In [ ]:
def find_neighbors(tract_id, df_census):
  poly_tract = Polygon(tracts_dict[tract_id][1])
  neighbors_fips = []
  for tract in tracts_dict:
    if tract_id == tract:
      continue
    poly_curr = Polygon(tracts_dict[tract][1])
    if poly_tract.touches(poly_curr):
      neighbors_fips.append(tract)
  neighbors_idx = []
  FIPS_List = df_census['Geo_FIPS']
  for fips in neighbors_fips:
    f_result = np.where(FIPS_List == fips)
    neighbors_idx.append(f_result[0][0])
  return neighbors_idx

In [ ]:
df_census_missing_na = df_census_missing.isna()

for ind, row in df_census_missing.iterrows(): # fill missing values based on mean of neighbors
  tract_id = row['Geo_FIPS']
  for col, val in row.iteritems():
    if df_census_missing_na[col][ind]:
      neighbors = find_neighbors(tract_id, df_census_missing)
      vals = []
      for neighbor in neighbors:
        vals.append(df_census_missing[col][neighbor])
      new_val = np.nanmean(np.array(vals))
      df_census_missing[col][ind] = new_val

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: Mean of empty slice
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [ ]:
# print(df_census_missing.isna().sum())

In [ ]:
df_census_missing_sorted = df_census_missing.sort_values('Geo_FIPS')
df_census_filled = df_census_missing_sorted.fillna(method='ffill')

In [ ]:
# print(df_census_filled.isna().sum())

## Download all Census Data

In [ ]:
# print(df_census_filled.columns)

Index(['Geo_FIPS', 'Geo_TRACT', 'r_concentrated_disadvantage', 'a_total_cars',
       'r_commute_time', 'r_median_household_income',
       'r_average_household_size', 'r_population_density',
       'r_median_house_value', 'a_total_population', 'a_pop_below_18',
       'a_pop_between_18_34', 'a_pop_between_34_65', 'a_pop_over_65',
       'a_occupied_households', 'a_family_households',
       'a_nonfamily_households', 'a_householder_less_than_high_school',
       'a_householder_high_school_degree', 'a_householder_some_college',
       'a_householder_bachelors_degree', 'a_older_than_3_in_school',
       'a_older_than_3_not_in_school', 'a_not_high_school_grad',
       'a_high_school_grad', 'a_employed', 'a_unemployed',
       'a_not_in_labor_force', 'a_30_49_percent_income_to_rent',
       'a_over_50_percent_income_to_rent', 'a_commuters_by_car_truck_van',
       'a_commuters_by_car_truck_van_alone',
       'a_commuters_by_car_truck_van_carpool',
       'a_children_living_with_single_pare

In [ ]:
from google.colab import files
df_census_filled.to_csv('cleaned_census_data_county.csv') 
files.download('cleaned_census_data_county.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Mapping Cars per Home

I had fun making the previous map so let's make another one :)

In [ ]:
df_census_copy = remove_fips_extras(df_census)
df_census_copy['Geo_TRACT'] = [str(x) for x in df_census_copy['Geo_TRACT']]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
import folium
import json
from folium import plugins

la_tracts_url = 'http://boundaries.latimes.com/1.0/boundary-set/census-tracts-2012/?format=geojson'

In [ ]:
la_map = folium.Map(location=[34.0522,-118.2437], tiles='Stamen Toner', zoom_start=9)
folium.Choropleth(
    geo_data=la_tracts_url, 
    data=df_census_filled, 
    columns=['Geo_FIPS', 'a_total_cars'], 
    key_on='feature.properties.name',
    fill_color='YlGnBu', 
    fill_opacity=1,
    line_opacity=.2
).add_to(la_map)

In [ ]:
la_map

# **Charger Dataframe Cleanup**
### Amber Guo and Dan Rabinovich

## Address Cleanup

### Amber Guo

In [ ]:
!pip install opencage
import pandas as pd
from opencage.geocoder import OpenCageGeocode
from tqdm.notebook import tqdm

     |████████████████████████████████| 61kB 3.6MB/s 
     |████████████████████████████████| 2.6MB 8.0MB/s 


In [ ]:
# get addresses of chargers in flo dataset
url = 'https://raw.githubusercontent.com/amberguo/ev-charger/main/Flo_Parks_Utilization_Report_addresses.csv'
# read in csv containing addresses
chargers_address = pd.read_csv(url, usecols=['Station','Park'])

# clean up station id
chargers_address['Station'] = chargers_address['Station'].str.split("\"").str[1]
# clean up address
chargers_address['Address'] = chargers_address['Park'].str.split("|").str[3]
chargers_address['Address'] = chargers_address['Address'].str[1:-1] + ", Los Angeles, CA"
chargers_address = chargers_address.drop(columns=['Park'])

print(chargers_address)
# convert address to lat/long
key = '868dedf036a34baf95772c88fbd4dbc5'
geocoder = OpenCageGeocode(key)


lats, lons = ([], []) 
for addr in tqdm(chargers_address['Address']):
  results = geocoder.geocode(addr)
  lat = results[0]['geometry']['lat']
  lng = results[0]['geometry']['lng']
  lats.append(lat)
  lons.append(lng)

chargers_address['lat'] = lats
chargers_address['lon'] = lons

       Station                                            Address
0    AUI-10372  2316 W Martin Luther King Jr Blvd, Los Angeles...
1    AUI-10263                    1231 S Hill St, Los Angeles, CA
2    AUI-10254                  942 S Crocker St, Los Angeles, CA
3    AUI-10265                5764 S Vermont Ave, Los Angeles, CA
4    AUI-10218                  7150 N Shoup Ave, Los Angeles, CA
..         ...                                                ...
172  AUI-10358                 13214 Moorpark St, Los Angeles, CA
173  AUI-10361                   11048 Huston St, Los Angeles, CA
174  AUI-10362                 998 S Hobart Blvd, Los Angeles, CA
175  AUI-10359               10919 S Figueroa St, Los Angeles, CA
176  AUI-10261             729 N North Spring St, Los Angeles, CA

[177 rows x 2 columns]


NameError: ignored

## Converting Latitude and Longitude to Census Tract and Zipcode

### Dan Rabinovich


In [ ]:
import urllib, json, requests
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
import geopy

In [ ]:
stations_url = 'https://github.com/amberguo/ev-charger/blob/main/clean_flo_charger.csv?raw=true'
chargers_address = pd.read_csv(stations_url, encoding = "ISO-8859-1")
chargers_address['lat'] = chargers_address['Latitude']
chargers_address['lon'] = chargers_address['Longitude']
chargers_address.drop(columns = ['Longitude', 'Latitude'], inplace = True)

In [ ]:
def get_zipcode(df, geolocator, lat_field, lon_field):
  location = geolocator.reverse((df['lat'], df['lon']))
  try:
    return location.raw['address']['postcode']
  except:
    return np.nan

geolocator = geopy.Nominatim(user_agent='my-application')
tqdm.pandas()
zipcodes = chargers_address.progress_apply(get_zipcode, axis=1, geolocator=geolocator, lat_field='lat', lon_field='lon')
chargers_address['zipcode'] = zipcodes

In [ ]:
# print(chargers_address)
# print(chargers_address[:1])

   Unnamed: 0    Station  Total Sessions  ...       lat         lon  zipcode
0           0  AUI-10372               5  ...  34.01072 -118.318881    90008

[1 rows x 7 columns]


In [ ]:
# %%time

def api_call(row): # This is from Prathik
  lat = row['lat'] # This returns the value in 'lat' for that row
  lon = row['lon']
  URL = "https://geo.fcc.gov/api/census/block/find?format=json&latitude=" + str(lat) + "&longitude=" + str(lon)
  # Input: https://geo.fcc.gov/api/census/area?lat=34.022350&lon=-118.285118&format=json
  with urllib.request.urlopen(URL) as url:
    data = json.loads(url.read().decode())
  try:
    return int(data['Block']['FIPS'])
  except TypeError:
    return np.nan

block_arr = []
for ind, row in tqdm(chargers_address.iterrows(), total = chargers_address.shape[0]):
  block_FIPS = api_call(row)
  block_arr.append(block_FIPS)

chargers_address['block_FIPS'] = block_arr
cols_to_keep = ['Station', 'Address', 'lat', 'lon', 'zipcode', 'block_FIPS']
chargers_address = chargers_address[cols_to_keep]
chargers_address['Geo_FIPS'] = ['0' + str(elem) for elem in chargers_address['block_FIPS']]
chargers_address['Geo_FIPS'] = [str(elem)[:-4] for elem in chargers_address['Geo_FIPS']]
chargers_address.drop(columns = 'block_FIPS', inplace = True)

print(chargers_address)



       Station  ...     Geo_FIPS
0    AUI-10372  ...  06037234000
1    AUI-10263  ...  06037207900
2    AUI-10254  ...  06037226002
3    AUI-10265  ...  06037232700
4    AUI-10218  ...  06037134303
..         ...  ...          ...
172  AUI-10358  ...  06037143500
173  AUI-10361  ...  06037125401
174  AUI-10362  ...  06037213201
175  AUI-10359  ...  06037241202
176  AUI-10261  ...  06037207102

[177 rows x 6 columns]


In [ ]:
# Now to download this
from google.colab import files
chargers_address.to_csv('addresses.csv') 
files.download('addresses.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>